In [8]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time
sys.path.append(('/home/ak/Research/PaperCode/stylised_facts'))
sys.path.append('/home/ak/Research/PaperCode/stylised_facts')
import stylised_facts_data_utilities as sfd_utils
import lob_for_futures as lobfut
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from multiprocessing import Pool, freeze_support
import pickle
from multiprocessing import Pool

scaler = MinMaxScaler()
standard_scaler = StandardScaler()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
## data files
laptop_OS_folder = '/media/ak/T7/FuturesDataSemiProcessed'
LaCie_ProcessedData = '/media/ak/LaCie/ProcessedSampledData/'
# returns_data = '/media/ak/T7/August11th2022Experiments/Returns/'
t7 = '/media/ak/T7/'
t7_folder = os.path.join(t7, 'FuturesDataSemiProcessed')
# june_ext = os.path.join(t7, 'June4th2022Experiments')
# returns_data = [f for f in os.listdir(june_ext) if '_returns' in f]
experimentsLocation = '/media/ak/T7/August11th2022Experiments/'

# here i start with RX1 to do all the experiments in one go
symbols = os.listdir(laptop_OS_folder)


# symbol_test_folder = os.path.join(laptop_OS_folder, symbols[0])
# rx_folder = os.path.join(laptop_OS_folder, 'RX1')
# du_folder = os.path.join(t7_folder,  'DU1')                                                                       , 'FB1')
# make this a bit more dynamic to take any function in here
# files = os.listdir(du_folder)



In [9]:

def produce_experiment_data(chosen_df):
    """
    # this is somewhat unit-tested in the August 7th 2022 notebook
    # function to produce dataframes for experiments
    # input: chosen df - this is a dataframe that we apply the microstructure features and the vol estimation features
    # output: experiment - dataframe with the features we want experiments for
    # written in August 2022
    # re-write it as part of the _init_ file

    """
    chosen_df_micro = lobfut.apply_micro_structure_features(chosen_df)  # get micro structure df
    vol_class = lobfut.volatilityEstimation(chosen_df)  # get the vol class

    # features I need: micro_price changes / vols /skews /etc:

    experiment_df = chosen_df_micro.loc[:, ['micro_price', 'price_imbalance',
                                            'pct_change_micro_price', 'weighted_activity_spread', ]]
    experiment_df['GK_vol'] = pd.Series(
        list(vol_class.garmanKlass(5)))  # get a sample of Garman - Klass resampled for 5 clicks
    experiment_df['arrival_rates'] = pd.Series(
        vol_class.arrival_rates().reshape(vol_class.arrival_rates().shape[0], )).replace([np.inf, -np.inf],
                                                                                         0).fillna(0)
    X = experiment_df.pct_change_micro_price.replace([np.inf, -np.inf], 0).values.reshape(-1, 1)
    norm_scaler = StandardScaler().fit(X)  # normalised scaling by mean and std
    min_max_scaler = MinMaxScaler().fit(X)  # min max scaling
    try:
        experiment_df['returns_normalised'] = norm_scaler.transform(
            X)  # use this format to get rid of the prior issues
        experiment_df['returns_mix_max'] = min_max_scaler.transform(X)  # get both issues of
        rs, rk = vol_class.realised_skewness_kurtosis()  # get skew/kurt

        experiment_df['skew'] = pd.Series(list(rs))
        experiment_df['kurt'] = pd.Series(list(rk))

        experiment_df['median_traded_volume'] = chosen_df_micro[['total_traded_volume_open',
                                                                 'total_traded_volume_high',
                                                                 'total_traded_volume_low',
                                                                 'total_traded_volume_close']].quantile(0.5, axis=1)
        experiment_df['jumps_test'] = pd.Series(vol_class.jumps_test(rollingWindow=5, sampling_param=0))
        experiment_df['relz_var'] = pd.Series(vol_class.realised_variance(rollingWindow=5))
        experiment_df['trip_quart'] = pd.Series(vol_class.tripower_quarticity(rollingWindow=5, sampling_param=0))

        experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up
    except ValueError:
        print("error")
        pass
    experiment_df = experiment_df.replace([np.inf, -np.inf], 0).fillna(0)  # final clean up

    return experiment_df



In [14]:
!pip install fathon

  Using cached fathon-1.2-cp36-cp36m-manylinux2010_x86_64.whl (23.5 MB)
  Using cached Cython-0.29.32-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (2.0 MB)


In [17]:

# if __name__ == '__main__':
#     """
#     function that takes the index of a file, that index corresponds to a point in a list of files that are already
#     aligned between trades and quotes and have been information clock processed
#     then applies the extraction of features
#     """

symbol = 'RX1'
def produce_and_dump(files_idx_, symbol_):
    symbol = symbol_  # and this
    symbol_folder_path = os.path.join(t7_folder, str(symbol))
    all_files = os.listdir(symbol_folder_path)
    files = [f for f in all_files if str('Returns_') not in f]

    choice_bar = 'dollar'  # change this
    date_idx = files[files_idx_].split(".")[0]
    print(date_idx)

    idx_file_path = os.path.join(symbol_folder_path,
                                 files[files_idx_])  # the input here needs to be dynamic not du_folder or rx_folder
    choice_df = pd.read_pickle(idx_file_path)[date_idx][choice_bar]
    exp_df = produce_experiment_data(choice_df)
    pickle_out_returns = os.path.join(experimentsLocation, "".join(
        (str(symbol) + "_" + str(choice_bar) + "_" + str(date_idx) + "_exp_df.pkl")))
    pickle.dump(exp_df, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    print('saved:', pickle_out_returns)


In [27]:
symbol = 'G_1'  # and this
symbol_folder_path = os.path.join(t7_folder, str(symbol))
all_files = os.listdir(symbol_folder_path)
files =[f for f in all_files if str('Returns_') not in f]

In [28]:
files

['20180716.pkl',
 '20180625.pkl',
 '20180509.pkl',
 '20180829.pkl',
 '20180815.pkl',
 '20180515.pkl',
 '20180711.pkl',
 '20180614.pkl',
 '20180601.pkl',
 '20180531.pkl',
 '20180418.pkl',
 '20180925.pkl',
 '20180626.pkl',
 '20180427.pkl',
 '20180522.pkl',
 '20180607.pkl',
 '20181003.pkl',
 '20180620.pkl',
 '20181001.pkl',
 '20180803.pkl',
 '20180416.pkl',
 '20180814.pkl',
 '20180918.pkl',
 '20180919.pkl',
 '20180924.pkl',
 '20180618.pkl',
 '20180705.pkl',
 '20181023.pkl',
 '20180524.pkl',
 '20180622.pkl',
 '20181012.pkl',
 '20180514.pkl',
 '20181019.pkl',
 '20180809.pkl',
 '20181005.pkl',
 '20180912.pkl',
 '20180822.pkl',
 '20180615.pkl',
 '20180502.pkl',
 '20180730.pkl',
 '20180905.pkl',
 '20180503.pkl',
 '20180820.pkl',
 '20180530.pkl',
 '20181024.pkl',
 '20180920.pkl',
 '20180629.pkl',
 '20180704.pkl',
 '20180926.pkl',
 '20180723.pkl',
 '20180801.pkl',
 '20180430.pkl',
 '20181016.pkl',
 '20180612.pkl',
 '20180802.pkl',
 '20180731.pkl',
 '20180713.pkl',
 '20180518.pkl',
 '20181010.pkl

In [5]:


# pool = Pool(6)
# start_time = time.perf_counter()
# processes = [pool.apply_async(produce_and_dump, args=(files_idx_,)) for files_idx_ in range(0, 5)]
# result = [p.get() for p in processes]
# finish_time = time.perf_counter()
# print(f"Program finished in {finish_time - start_time} seconds")
# print(result)


In [18]:
produce_and_dump(1, 'RX1')

20180430
Applying micro-structure features
usd_volume_bucket
1783.0        06:01:08.367000
2674.0        06:01:08.372000
3566.0        06:01:08.378000
4457.0        06:01:08.379000
5349.0        06:01:08.397000
                   ...       
14017892.0    19:59:53.006000
14017893.0    19:59:55.151000
14017901.0    19:59:56.074000
14017917.0    20:00:00.052000
14017933.0           20:03:17
Name: TimeStamp_open, Length: 342223, dtype: object


/home/ak/anaconda3/envs/workresr/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ak/anaconda3/envs/workresr/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/ak/anaconda3/envs/workresr/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


saved: /media/ak/T7/August11th2022Experiments/RX1_dollar_20180430_exp_df.pkl


In [12]:
 # and this
files_idx_ =1
symbol_folder_path = os.path.join(t7_folder, str(symbol))
all_files = os.listdir(symbol_folder_path)
files = [f for f in all_files if str('Returns_') not in f]

choice_bar = 'dollar'  # change this
date_idx = files[files_idx_].split(".")[0]
print(date_idx)

idx_file_path = os.path.join(symbol_folder_path,
                             files[files_idx_])  # the input here needs to be dynamic not du_folder or rx_folder
choice_df = pd.read_pickle(idx_file_path)[date_idx][choice_bar]

20180430


In [ ]:
pd.read_pickle(idx_file_path)

In [13]:
choice_df

,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,type_open,type_high,type_low,type_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close
usd_volume_bucket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1783.0,5,5,5,5,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,BID,BID,BID,BID,158.62,158.62,158.62,158.62,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,7,7,7,7,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,ASK,ASK,ASK,ASK,158.64,158.64,158.64,158.64,562,562,562,562,TRADE,TRADE,TRADE,TRADE,158.64,158.64,158.64,158.64,06:01:08.367000,06:01:08.367000,06:01:08.367000,06:01:08.367000,562,562,562,562,562,562,562,562,21668000,21668000,21668000,21668000,89155.68,89155.68,89155.68,89155.68,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,158.631667,158.631667,158.631667,158.631667
2674.0,6,6,6,6,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,BID,BID,BID,BID,158.62,158.62,158.62,158.62,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,2018-04-30 06:01:08.372,7,7,7,7,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,ASK,ASK,ASK,ASK,158.64,158.64,158.64,158.64,562,562,562,562,TRADE,TRADE,TRADE,TRADE,158.64,158.64,158.64,158.64,06:01:08.372000,06:01:08.372000,06:01:08.372000,06:01:08.372000,562,562,562,562,562,562,562,562,21668000,21668000,21668000,21668000,89155.68,89155.68,89155.68,89155.68,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,158.630769,158.630769,158.630769,158.630769
3566.0,9,9,9,9,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,BID,BID,BID,BID,158.62,158.62,158.62,158.62,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,2018-04-30 06:01:08.378,7,7,7,7,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,ASK,ASK,ASK,ASK,158.64,158.64,158.64,158.64,562,562,562,562,TRADE,TRADE,TRADE,TRADE,158.64,158.64,158.64,158.64,06:01:08.378000,06:01:08.378000,06:01:08.378000,06:01:08.378000,562,562,562,562,562,562,562,562,21668000,21668000,21668000,21668000,89155.68,89155.68,89155.68,89155.68,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,158.628750,158.628750,158.628750,158.628750
4457.0,11,11,11,11,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,BID,BID,BID,BID,158.62,158.62,158.62,158.62,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,2018-04-30 06:01:08.379,7,7,7,7,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,2018-04-30 06:01:08.367,ASK,ASK,ASK,ASK,158.64,158.64,158.64,158.

In [21]:
os.listdir('/media/ak/T7/August11th2022Experiments/')

['Returns',
 'ExperimentOne',
 'ExperimentTwo',
 'mfdfaDataFrames',
 'RX1_dollar_20180430_exp_df.pkl',
 'RX1_dollar_20180709_exp_df.pkl',
 'RX1_tick_20180709_exp_df.pkl',
 'G_1_tick_20180716_exp_df.pkl',
 'G_1_tick_20180509_exp_df.pkl',
 'G_1_tick_20180625_exp_df.pkl',
 'test.pkl',
 'ExperimentInputFiles']

In [23]:
pd.read_pickle('/media/ak/T7/August11th2022Experiments/G_1_tick_20180626_exp_df.pkl')

,micro_price,price_imbalance,pct_change_micro_price,weighted_activity_spread,GK_vol,arrival_rates,returns_normalised,returns_mix_max,skew,kurt,median_traded_volume
0,123.428832,-22.548579,0.000000,0.014368,0.000306,69.0,0.000000,0.000000,0.000000,0.000000e+00,3.5
1,123.408958,-18.875743,-0.000141,0.010254,0.000300,100.0,-0.407084,0.924635,0.000000,0.000000e+00,1.0
2,123.402989,23.684391,0.000103,0.011605,0.000298,77.0,0.310532,0.940398,0.000000,0.000000e+00,3.0
3,123.419123,32.862695,0.000163,0.009378,0.000292,63.0,0.487651,0.944288,0.000000,0.000000e+00,3.0
4,123.437050,31.378126,0.000077,0.007672,0.000293,42.0,0.232248,0.938678,0.000000,0.000000e+00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2094,122.760590,23.751344,0.000011,0.008755,0.000000,0.0,0.038271,0.934417,0.000123,3.742895e-12,3.5
2095,122.755664,28.333497,-0.000005,0.011125,0.000000,0.0,-0.006382,0.933437,0.000123,3.740145e-12,4.0
2096,122.754843,-6.940314,-0.000120,0.016198,0.000000,0.0,-0.344605,0.926008,0.000124,3.767923e-12,2.5
2097,122.767283,-19.565326,0.000286,0.012866,0.000000,0.0,0.849793,0.952242,0.000184,1.203043e-11,2.0
